<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Toxic_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 71.3MB/s]
 41% 5.00M/12.1M [00:00<00:00, 43.9MB/s]
100% 12.1M/12.1M [00:00<00:00, 76.8MB/s]
 95% 260M/273M [00:01<00:00, 153MB/s]
100% 273M/273M [00:01<00:00, 143MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pickle
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate, CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Sequential, load_model, Model
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils import Sequence
from sklearn.model_selection import train_test_split

In [0]:
MAX_LEN = 220
BATCH_SIZE = 512
NUM_EPOCHS = 4
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 512
NUM_MODELS = 2

In [3]:
train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')
# preprocess text file
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def text_prepare(text):
    for p in punct:
        text = text.replace(p, ' ')
    return text.split()
train_text = [text_prepare(row) for row in tqdm(train['comment_text'])]
test_text = [text_prepare(row) for row in tqdm(test['comment_text'])]

100%|██████████| 97320/97320 [00:01<00:00, 53981.03it/s]


In [4]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(train),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(np.int) / 4
# loss_weight = 1.0 / weights.mean()
y_train = (train['target'].values >= 0.5).astype(np.int)
# y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values
del train
gc.collect()

7

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tqdm(train_text+test_text))
X_train = tokenizer.texts_to_sequences(tqdm(train_text))
X_test = tokenizer.texts_to_sequences(test_text)
del train_text, test_text, tokenizer
gc.collect()

100%|██████████| 1804874/1804874 [01:04<00:00, 27866.17it/s]


0

In [0]:
# X_train = np.load('drive/My Drive/Data/X_train.npy')
# X_test = np.load('drive/My Drive/Data/X_test.npy')
embedding_matrix = np.load('drive/My Drive/Data/crawl+glove.npy')

In [0]:
X_tr, X_val, y_tr, y_val, w_tr, w_val = train_test_split(X_train, y_train, weights, stratify=y_train, test_size=0.01, random_state=2019)
class TrainSeq(Sequence):

    def __init__(self, batch_size=512, percentile=95):
        self.x, self.y, self.w = X_tr, y_tr, w_tr
        self.batch_size = batch_size
        self.percentile = percentile

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_w = self.w[idx * self.batch_size:(idx + 1) * self.batch_size]
        length = min(int(np.percentile([len(o) for o in batch_x], self.percentile)), MAX_LEN)
        batch_x = np.array(pad_sequences(batch_x, length, padding='post'))
        return {model.input_names[0]: batch_x, model.input_names[1]: batch_w}, batch_y, batch_w
class ValSeq(Sequence):

    def __init__(self, batch_size=512, percentile=95):
        self.x, self.y, self.w = X_val, y_val, w_val
        self.batch_size = batch_size
        self.percentile = percentile

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_w = self.w[idx * self.batch_size:(idx + 1) * self.batch_size]
        length = min(int(np.percentile([len(o) for o in batch_x], self.percentile)), MAX_LEN)
        batch_x = np.array(pad_sequences(batch_x, length, padding='post')).reshape(-1, length)
        return {model.input_names[0]: batch_x, model.input_names[1]: batch_w}, batch_y, batch_w
      
class PredSeq(Sequence):

    def __init__(self, batch_size=2048, percentile=95):
        self.x = X_test
        self.batch_size = batch_size
        self.percentile = percentile

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        length = min(int(np.percentile([len(o) for o in batch_x], self.percentile)), MAX_LEN)
        batch_x = np.array(pad_sequences(batch_x, length, padding='post'))
        batch_w = np.ones(self.batch_size)
        return {model.input_names[0]: batch_x, model.input_names[1]: batch_w}

In [0]:
def auc(weights):
    def metric(y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred, weights=weights)[1]
        K.get_session().run(tf.local_variables_initializer())
        return auc
    return metric
# def custom_loss(y_true, y_pred):
#     return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]
def build_model(embedding_matrix):

    words = Input(shape=(None, ))
    weight = Input(shape=(1,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNGRU(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([GlobalMaxPooling1D()(x), GlobalAveragePooling1D()(x), ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
#     aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)

    model = Model(inputs=[words, weight], outputs=result)
    model.compile(loss='binary_crossentropy',
                  # loss_weights=[loss_weight, 1.0],
                  metrics=[auc(weight)],
                  optimizer='adam')

    return model



In [0]:
checkpoint_predictions = []
weights = []
for model_idx in range(NUM_MODELS):
    model = build_model(embedding_matrix)
    for global_epoch in range(4):
        model.fit_generator(TrainSeq(), 
                            validation_data=ValSeq(), 
                            verbose=1,
                            epochs=1,
                            callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))]
                           )
#             model.fit(
#                 [X_tr, y_tr[:, 1]], y_tr,
#                 batch_size=BATCH_SIZE,
#                 epochs=1,
#                 verbose=1,
#                 validation_data=([X_val, y_val[:, 1]], y_val),
#                 callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))]
#             )
        checkpoint_predictions.append(model.predict_generator(PredSeq()).flatten())
        weights.append(2 ** global_epoch)
    del model
    gc.collect()

preds = np.average(checkpoint_predictions, weights=weights, axis=0)
# return preds

Epoch 1/1
3490/3490 [==============================] - 729s 209ms/step - loss: 0.0675 - metric: 0.8964 - val_loss: 0.0608 - val_metric: 0.9237
Epoch 1/1
3490/3490 [==============================] - 732s 210ms/step - loss: 0.0583 - metric: 0.9308 - val_loss: 0.0554 - val_metric: 0.9354
Epoch 1/1
3490/3490 [==============================] - 732s 210ms/step - loss: 0.0548 - metric: 0.9390 - val_loss: 0.0556 - val_metric: 0.9418
Epoch 1/1
1468/3490 [===========>..................] - ETA: 7:03 - loss: 0.0523 - metric: 0.9430